# Lab 1 Shkadarevich Dmitry - Branch and Bounds method

# Formulation of the problem

Find the optimal integer solution to the LP problem

<img src="algo.png">

# Code

In [27]:
import numpy as np
import queue
import math
import copy
from scipy import optimize

In [3]:
def is_int(x): #check if x is integer
    return round(x, 3).is_integer()

In [36]:
class LPTask:
    '''
    Linear programming task class

    Attributes:
    cost_function(c) : 1-D array
        The coefficients of the linear objective function to be minimized.
    
    A_ub : 2-D array
        The inequality constraint matrix. Each row of A_ub specifies the 
        coefficients of a linear inequality constraint on x.
    
    b_ub : 1-D array
        The inequality constraint vector. Each element represents an 
        upper bound on the corresponding value of A_ub @ x.
    
    bounds : sequence
        A sequence of (min, max) pairs for each element in x, defining 
        the minimum and maximum values of that decision variable.
    '''
    def __init__(self, c, A_ub, b_ub, bounds):
        self.solution = optimize.linprog(c, A_ub, b_ub, bounds=bounds)
        self.x = np.around(self.solution.x,decimals=3) # x optimal
        self.fun = self.solution.fun # cost function value
        self.success = self.solution.success # is solution feasible?
        self.bounds = bounds

    def is_explored(self):  # Check if all x values are integer
        return all(is_int(x_i) for x_i in self.x) 

    def __str__(self):
        return self.solution.__str__()

    def get_float_x(self):
         for i,x in enumerate(self.x):
            if not is_int(x):
                return i,x


class BnBSolver:  # Branch and bound method
    def __init__(self, initial_task):
        self.queue = [initial_task] # Queue of all LP subtasks
        self.result = None
        self.upper_bound = np.inf # Upper bound of cost function

    def solve(self):  # Returns solved LP task with optimal integer solution
        while self.queue:
            curr_task = self.queue.pop()
            if not curr_task.success:
                continue
            if curr_task.is_explored():
                if curr_task.fun <= self.upper_bound: 
                    self.result = curr_task
                    self.upper_bound = curr_task.fun
                else:
                    continue
            else:
                self.create_branches(curr_task)
        return self.result

    def create_branches(self, task):
        i,float_x = task.get_float_x()

        x_upper_bound = math.ceil(float_x)
        x_lower_bound = math.floor(float_x)

        # LP1(left) - x[i]<=lower_bound 
        bounds_left = copy.deepcopy(task.bounds)
        bounds_left[i][1] = x_lower_bound
        lp_left = LPTask(c,A_ub,b_ub,bounds_left)
        self.queue.append(lp_left)
        
        # LP2(right) - x[i]>=upper_bound 
        bounds_right = copy.deepcopy(task.bounds)
        bounds_right[i][0] = x_upper_bound
        lp_right = LPTask(c,A_ub,b_ub,bounds_right)
        self.queue.append(lp_right)


# Test cases

<img src="1.png">
<img src="1_res.png">

In [39]:
c = np.array([-5,-4])
A_ub = np.array([[1,1],[10,6]])
b_ub = np.array([5,45])
bounds = [[0,np.inf],[0,np.inf]]

lp0 = LPTask(c,A_ub,b_ub,bounds)

branch_and_bound_solver = BnBSolver(lp0)
print(branch_and_bound_solver.solve())

     con: array([], dtype=float64)
     fun: -22.99999985484144
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([3.25218332e-08, 3.00000026e+00])
  status: 0
 success: True
       x: array([2.99999998, 1.99999998])


<img src="2.png">

In [42]:
c = np.array([-7,-9])
A_ub = np.array([[-1,3],[7,1]])
b_ub = np.array([6,35])
bounds = [[0,np.inf],[0,np.inf]]

lp0 = LPTask(c,A_ub,b_ub,bounds)

branch_and_bound_solver = BnBSolver(lp0)
print(branch_and_bound_solver.solve())

     con: array([], dtype=float64)
     fun: -54.99999999726958
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([1., 4.])
  status: 0
 success: True
       x: array([4., 3.])


<img src="3.png">

In [43]:
c = np.array([-2,-3])
A_ub = np.array([[3,4],[2,5]])
b_ub = np.array([24,22])
bounds = [[0,np.inf],[0,np.inf]]

lp0 = LPTask(c,A_ub,b_ub,bounds)

branch_and_bound_solver = BnBSolver(lp0)
print(branch_and_bound_solver.solve())

     con: array([], dtype=float64)
     fun: -15.999999999225022
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([1., 2.])
  status: 0
 success: True
       x: array([5., 2.])
